# Data Quality Assessment

## Importing The Necessary Libraries

In [3]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 
import seaborn as sns

## Importing Our Dataset

In [4]:
kpmg_transactions = pd.read_excel(r'C:\Users\USER\Documents\Dataset\KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 'Transactions')

new_customer_List = pd.read_excel(r'C:\Users\USER\Documents\Dataset\KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 'NewCustomerList')\

customerdemographic = pd.read_excel(r'C:\Users\USER\Documents\Dataset\KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 'CustomerDemographic')

customer_address = pd.read_excel(r'C:\Users\USER\Documents\Dataset\KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 'CustomerAddress')

#### Setting the display options of the dataframes rows and columns

In [5]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [6]:
new_customer_List.drop(new_customer_List.iloc[:, 16:21], axis=1, inplace = True)

#  kpmg_transactions

In [7]:
kpmg_transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Checking Dataset Rows And Columns

In [8]:
kpmg_transactions.shape

(20000, 26)

## Data Cleaning And Transformation

#### Dropping The Unnecessary Columns and rows From Our Transactions Dataset

In [9]:
kpmg_transactions.drop(kpmg_transactions.iloc[:, 13:], axis=1, inplace = True)

In [10]:
kpmg_transactions.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03


In [11]:
kpmg_transactions.order_status.value_counts()

Approved     19821
Cancelled      179
Name: order_status, dtype: int64

Present in the dataset are 179 cancelled transactions. These set of transactions are irrelevant to our analysis. So, we would be dropping all rows with cancelled transactions.

In [12]:
kpmg_transactions=kpmg_transactions[kpmg_transactions['order_status'] == 'Approved']

#### Transforming The Online Order Data

In [13]:
kpmg_transactions['online_order']=kpmg_transactions['online_order'].replace({0.0:'No', 1.0:'Yes'})

#### Creating The Profit Column In Our Dataset

In [14]:
profit = kpmg_transactions['list_price']-kpmg_transactions['standard_cost']
kpmg_transactions.insert(12,'profit',profit)

## Checking For Duplicates

In [15]:
kpmg_transactions.duplicated().sum()

0

### Checking For Columns With missing values

In [16]:
kpmg_transactions.isnull().sum()[(kpmg_transactions.isnull().sum()) > 0]

online_order               354
brand                      196
product_line               196
product_class              196
product_size               196
standard_cost              196
profit                     196
product_first_sold_date    196
dtype: int64

### Percentage Of  Data Missing From Each Column

In [17]:
missing_values = kpmg_transactions[kpmg_transactions.isnull().any(axis=1)].shape[0]
data_rows = kpmg_transactions.shape[0]

In [18]:
kpmg_transactions.isnull().sum()[(kpmg_transactions.isnull().sum()) > 0]/data_rows*100

online_order               1.785985
brand                      0.988850
product_line               0.988850
product_class              0.988850
product_size               0.988850
standard_cost              0.988850
profit                     0.988850
product_first_sold_date    0.988850
dtype: float64

Online order has the most missing values, with approximately 1.8 percent of its values missing from the column. While the other seven other columns accounts for less than one percent of their values missing in their respective columns.

### Percentage Of The Total Missing Values From Our Dataset

In [17]:
missing_values

548

In [18]:
percent = (missing_values/data_rows)*100
percent = round(percent,2)

In [19]:
missing_values

548

In [20]:
print((f'{percent} percent of our data are missing from our data set.\nThat is to break it down, our dataset has {kpmg_transactions.shape[0]} rows and {kpmg_transactions.shape[1]} columns.\n8 out of the 14 rows have at least a missing value.\nThe total rows with at least a missing value is {missing_values}, which is about 2.775 percent of the total number of rows.\nSo, we would be dropping these {missing_values} as they represent less than 5 percent of our dataset and not discarding them might skew our analytical process.') )

2.76 percent of our data are missing from our data set.
That is to break it down, our dataset has 19821 rows and 14 columns.
8 out of the 14 rows have at least a missing value.
The total rows with at least a missing value is 548, which is about 2.775 percent of the total number of rows.
So, we would be dropping these 548 as they represent less than 5 percent of our dataset and not discarding them might skew our analytical process.


### Dropping Missing Values

In [21]:
kpmg_transactions.dropna(inplace = True)

In [19]:
kpmg_transactions.shape

(19821, 14)

### Saving Our Dataset

In [23]:
kpmg_transactions.to_csv('kpmg_transactions.csv', index = False)

#  new_customer_List

In [24]:
new_customer_List.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


#### Checking Dataset Rows And Columns

In [25]:
new_customer_List.shape

(1000, 18)

### Data Cleaning And Transformation

### Checking For Duplicates

In [26]:
new_customer_List.duplicated().sum()

0

### Number Of Missing Rows From Every Column With Missing Data

In [27]:
new_customer_List.isnull().sum()[new_customer_List.isnull().sum() > 0]

last_name                 29
DOB                       17
job_title                106
job_industry_category    165
dtype: int64

### Percentage Of Missing Rows From Every Column With Missing Data

In [28]:
rows = new_customer_List.shape[0]
null_rows = new_customer_List[new_customer_List.isnull().any(axis = 1)].shape[0]

In [29]:
new_customer_List.isnull().sum()[(new_customer_List.isnull().sum()) > 0]/rows*100

last_name                 2.9
DOB                       1.7
job_title                10.6
job_industry_category    16.5
dtype: float64

### Percentage Of The Total Missing Values From Our Dataset

In [30]:
null_rows

285

In [31]:
percent = (null_rows/rows)*100
percent = round(percent,2)

In [32]:
print((f'{percent} percent of our data are missing from our data set.\nThat is to break it down, our dataset has {new_customer_List.shape[0]} rows and {new_customer_List.shape[1]} columns.\n4 out of the 18 rows have at least a missing value.\nThe total rows with at least a missing value is {null_rows}, which is about 28.5 percent of the total number of rows.\nSo, we would be dropping these {null_rows} rows as it might skew our analytical process.') )

28.5 percent of our data are missing from our data set.
That is to break it down, our dataset has 1000 rows and 18 columns.
4 out of the 18 rows have at least a missing value.
The total rows with at least a missing value is 285, which is about 28.5 percent of the total number of rows.
So, we would be dropping these 285 rows as it might skew our analytical process.


### Dropping Missing Values

In [33]:
new_customer_List.dropna(inplace = True)

In [34]:
new_customer_List.shape

(715, 18)

### Creating The Age Column

In [35]:
age = new_customer_List['DOB'].apply(lambda x:2017 - x.year)
new_customer_List.insert(5, 'age', age)

In [36]:
new_customer_List.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,60,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,47,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,43,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,38,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,52,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


### Data Types Of The Columns

In [37]:
new_customer_List.dtypes

first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
age                                             int64
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                          int64
address                                        object
postcode                                        int64
state                                          object
country                                        object
property_valuation                              int64
Rank                                            int64
Value                       

### Saving Our Dataset

In [38]:
new_customer_List.to_csv('kpmg_new_customers.csv', index = False)

# customerdemographic

In [40]:
customerdemographic.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


#### Dataset Rows And Columns

In [41]:
customerdemographic.shape

(4000, 13)

### Data Cleaning And Transformation

In [42]:
customerdemographic.duplicated().sum()

0

### Dropping Irrelevant Rows And Columns

#### Dropping The Default Column

In [43]:
del customerdemographic['default']
#customerdemographic.drop('default', axis =1)

#### Dropping Rows With Deceased Indicator as Yes

In [44]:
customerdemographic[customerdemographic['deceased_indicator']== 'Y']

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
752,753,Josy,St. Quentin,Female,82,1970-03-07,Food Chemist,Health,Affluent Customer,Y,Yes,6.0
3789,3790,Kurtis,Morson,Male,91,1959-05-31,Senior Editor,Retail,Mass Customer,Y,Yes,9.0


In [45]:
#Dropping The Above Rows

customerdemographic.drop([752, 3789], inplace = True)

#### Dropping Outliers

In [62]:
customerdemographic.sort_values('age', ascending = False).head(2)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
33,34,Jephthah,Bachmann,U,59,1843-12-21,174,Legal Assistant,IT,Affluent Customer,N,No,20.0
719,720,Darrel,Canet,Male,67,1931-10-23,86,Recruiting Manager,Retail,Affluent Customer,N,No,6.0


In [63]:
#Dropping of the row with the customer of age 174

customerdemographic.drop([33], inplace = True)

In [65]:
# Confirming Our maximum customer age again

customerdemographic.sort_values('age', ascending = False).head(2)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
719,720,Darrel,Canet,Male,67,1931-10-23,86,Recruiting Manager,Retail,Affluent Customer,N,No,6.0
1091,1092,Katlin,Creddon,Female,56,1935-08-22,82,VP Quality Control,Retail,Mass Customer,N,No,5.0


### Number Of Missing Rows From Every Column With Missing Data

In [99]:
customerdemographic.isnull().sum()[customerdemographic.isnull().sum() > 0]

last_name                125
DOB                       87
job_title                506
job_industry_category    656
default                  302
tenure                    87
dtype: int64

### Percentage Of Missing Rows From Every Column With Missing Data

In [47]:
nulls = customerdemographic[customerdemographic.isnull().any(axis=1)].shape[0]
data_rows = customerdemographic.shape[0]

In [48]:
customerdemographic.isnull().sum()[(customerdemographic.isnull().sum()) > 0]/data_rows*100

last_name                 3.126563
DOB                       2.176088
job_title                12.656328
job_industry_category    16.408204
tenure                    2.176088
dtype: float64

### Percentage Of The Total Missing Values From Our Dataset

In [49]:
nulls

1220

In [50]:
percentage = (nulls/data_rows)*100
percentage = round(percentage,2)

In [53]:
print((f'{percentage} percent of our data are missing from our data set.\nThat is to break it down, our dataset has {customerdemographic.shape[0]} rows and {customerdemographic.shape[1]} columns.\n5 out of the 12 rows have at least a missing value.\nThe total rows with at least a missing value is {null_rows}, which is about {percentage} percent of the total number of rows.\nSo, we would be dropping these {null_rows} rows as it might skew our analytical process.') )

30.52 percent of our data are missing from our data set.
That is to break it down, our dataset has 3998 rows and 12 columns.
5 out of the 12 rows have at least a missing value.
The total rows with at least a missing value is 285, which is about 30.52 percent of the total number of rows.
So, we would be dropping these 285 rows as it might skew our analytical process.


### Dropping Missing Values

In [56]:
customerdemographic.dropna(inplace = True)

In [66]:
customerdemographic.shape

(2777, 13)

### Creating The Age Column

In [58]:
age2 = customerdemographic['DOB'].apply(lambda x: 2017 - x.year)
customerdemographic.insert(6, 'age', age2)

### Setting The Gender Column

In [69]:
customerdemographic['gender']=customerdemographic['gender'].replace({'F':'Female','Femal':'Female','M':'Male'})

### Checking The Columns Datatype

In [67]:
customerdemographic.dtypes

customer_id                                     int64
first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
age                                             int64
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                        float64
dtype: object

### Saving The Dataset

In [75]:
customerdemographic.to_csv('kpmg_customer_demographic.csv', index = False)

# customer_address

In [78]:
customer_address.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [77]:
customer_address.shape

(3999, 6)

### Checking Customer Datatype

In [106]:
customer_address.dtypes

customer_id            int64
address               object
postcode               int64
state                 object
country               object
property_valuation     int64
dtype: object

### Checking For Duplicated Rows

In [79]:
customer_address.duplicated('customer_id').sum()

0

### Checking For Missing Values

In [80]:
customer_address.isnull().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

In [111]:
customer_address.country.value_counts()

Australia    3999
Name: country, dtype: int64

### Setting Our State Column To Have Consistent Precise Values

In [81]:
customer_address.state.unique()  #checking our state values

array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object)

In [84]:
new = {'New South Wales':'NSW', 'Victoria':'VIC'}   #mapping out our state values with more precise values

customer_address['state'].replace(new, inplace = True)  #replacing the state values with the mapped out values

customer_address.state.unique()  #confirming the consistency of our state values

array(['NSW', 'QLD', 'VIC'], dtype=object)

### Saving The Dataset

In [91]:
customer_address.to_csv('kpmg_customer_address.csv', index = False)

### Missing Customer Id's From Datasets Prior Or Post Data Cleaning

### List of customers id's from customer demographics not found in customer address

In [85]:
customerdemographic[~customerdemographic['customer_id'].isin(customer_address['customer_id'])]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
2,3,Arlin,Dearle,Male,61,1954-01-20,63,Recruiting Manager,Property,Mass Customer,N,Yes,15.0
9,10,Fiorenze,Birdall,Female,49,1988-10-11,29,Senior Quality Engineer,Financial Services,Mass Customer,N,Yes,20.0


## List of customers id's from customer address not found in customer demographics

In [86]:
customer_address[~customer_address['customer_id'].isin(customerdemographic['customer_id'])]

,customer_id,address,postcode,state,country,property_valuation
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,NSW,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9
5,7,4 Delaware Trail,2210,NSW,Australia,9
6,8,49 Londonderry Lane,2650,NSW,Australia,4
...,...,...,...,...,...,...
3994,3999,1482 Hauk Trail,3064,VIC,Australia,3
3995,4000,57042 Village Green Point,4511,QLD,Australia,6
3996,4001,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,4002,8194 Lien Street,4032,QLD,Australia,7


## List of customers id's from transactions not found in customer demographics

In [87]:
kpmg_transactions[~kpmg_transactions['customer_id'].isin(customerdemographic['customer_id'])]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,profit,product_first_sold_date
4,5,78,787,2017-10-01,Yes,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,1055.82,2015-08-10
10,11,5,1986,2017-01-17,No,Approved,Trek Bicycles,Mountain,low,medium,574.64,459.71,114.93,2011-08-29
16,17,79,2426,2017-04-03,No,Approved,Norco Bicycles,Standard,medium,medium,1555.58,818.01,737.57,2004-08-07
24,25,89,2822,2017-06-11,No,Approved,WeareA2B,Touring,medium,large,1362.99,57.74,1305.25,1993-04-20
25,26,64,2596,2017-01-10,No,Approved,Trek Bicycles,Standard,medium,large,1469.44,596.55,872.89,2015-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19988,19989,0,714,2017-04-27,Yes,Approved,Norco Bicycles,Standard,low,medium,363.01,290.41,72.60,2003-01-05
19991,19992,11,1374,2017-09-14,No,Approved,Giant Bicycles,Standard,high,medium,1274.93,764.96,509.97,2007-08-04
19992,19993,13,5,2017-04-28,No,Approved,Solex,Standard,medium,medium,1163.89,589.27,574.62,2009-03-08
19993,19994,77,2618,2017-12-23,No,Approved,Norco Bicycles,Road,medium,large,1240.31,795.10,445.21,2011-01-10


## List of customers id's from transactions not found in customer address

In [88]:
kpmg_transactions[~kpmg_transactions['customer_id'].isin(customer_address['customer_id'])]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,profit,product_first_sold_date
117,118,20,22,2017-03-21,No,Approved,Trek Bicycles,Standard,medium,small,1775.81,1580.470000,195.340000,2010-05-05
3479,3480,32,22,2017-09-18,No,Approved,Giant Bicycles,Standard,medium,medium,642.70,211.370000,431.330000,2002-03-22
3503,3504,38,22,2017-07-16,Yes,Approved,Solex,Standard,medium,medium,1577.53,826.510000,751.020000,2011-03-16
4564,4565,5,10,2017-08-26,No,Approved,Trek Bicycles,Mountain,low,medium,574.64,459.710000,114.930000,2004-09-28
4600,4601,65,22,2017-05-29,Yes,Approved,WeareA2B,Standard,medium,medium,1807.45,778.690000,1028.760000,2015-05-21
5597,5598,85,23,2017-12-20,No,Approved,WeareA2B,Standard,medium,medium,1228.07,400.910000,827.160000,2015-08-10
5955,5956,21,10,2017-06-20,Yes,Approved,Solex,Standard,medium,large,1071.23,380.740000,690.490000,1993-05-26
7428,7429,54,23,2017-10-06,No,Approved,WeareA2B,Standard,medium,medium,1807.45,778.690000,1028.760000,2016-12-06
7950,7951,19,23,2017-08-03,Yes,Approved,Trek Bicycles,Mountain,low,medium,574.64,459.710000,114.930000,2007-08-04
8206,8207,53,22,2017-09-19,No,Approved,OHM Cycles,Standard,medium,medium,795.34,101.580000,693.760000,1997-02-09


## List of customers id's from customer address not found in transactions

In [89]:
customer_address[~customer_address['customer_id'].isin(kpmg_transactions['customer_id'])]

,customer_id,address,postcode,state,country,property_valuation
282,287,5583 Prairie Rose Trail,2765,NSW,Australia,8
847,852,641 Waywood Lane,2336,NSW,Australia,7
864,869,6712 Nova Crossing,2065,NSW,Australia,10
1368,1373,939 Hermina Drive,3198,VIC,Australia,8
2069,2074,0442 Debra Point,2281,NSW,Australia,7
...,...,...,...,...,...,...
3994,3999,1482 Hauk Trail,3064,VIC,Australia,3
3995,4000,57042 Village Green Point,4511,QLD,Australia,6
3996,4001,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,4002,8194 Lien Street,4032,QLD,Australia,7


## List of customers id's from customer demographics not found in transactions

In [90]:
customerdemographic[~customerdemographic['customer_id'].isin(kpmg_transactions['customer_id'])]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
286,287,Hannie,Wodham,Female,41,1985-05-06,32,Nuclear Power Engineer,Manufacturing,Affluent Customer,N,No,4.0
851,852,Andie,Bonney,Female,37,2000-11-04,17,Compensation Analyst,Financial Services,Affluent Customer,N,Yes,2.0
868,869,Addia,Abels,Female,7,1984-03-11,33,Account Representative I,Financial Services,High Net Worth,N,Yes,10.0
1372,1373,Shaylynn,Epsley,Female,3,1958-09-23,59,Director of Sales,Financial Services,Mass Customer,N,Yes,12.0
2875,2876,Swen,McGarahan,Male,68,1981-01-29,36,General Manager,Retail,High Net Worth,N,No,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3992,3993,Andi,Dumelow,Female,6,1974-12-05,43,Librarian,Entertainment,Mass Customer,N,No,10.0
3993,3994,Stephie,Byars,Female,5,1989-04-07,28,Structural Analysis Engineer,Manufacturing,Affluent Customer,N,No,12.0
3994,3995,Rusty,Iapico,Male,93,1975-12-12,42,Staff Scientist,Manufacturing,Mass Customer,N,Yes,14.0
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,42,VP Product Management,Health,Mass Customer,N,No,19.0
